In [25]:
import numpy as np 
from scipy.signal import convolve2d
import cv2

### Ejercicio 3

In [22]:
def convolution_2d(image1, image2):
    M1, N1 = image1.shape
    M2, N2 = image2.shape

    Mr = M1 + M2 - 1
    Nr = N1 + N2 - 1
    
    C = np.zeros((Mr, Nr))
    
    padded_image = np.zeros((M1 + 2*(M2-1),N2 + 2*(N2-1)))
    
    for i in range(M1):
        for j in range(N1): 
            padded_image[i + M2-1, j + N2-1] = image1[i,j]
    
    for i in range(M1):
        for j in range(N1):
            region = padded_image[i:i+M2, j:j+N2]
            C[i, j] = np.sum(region * image2)
    
    return C

In [23]:
x = np.array([[1, 2, 1], 
              [0, 1, 0], 
              [2, 1, 2]])

h1 = np.array([[0, 1], 
               [1, 0]])

h2 = np.array([[1, 0], 
               [0, 1]])

# Propiedad Conmutativa: x * h1 = h1 * x
conv_x_h1 = convolve2d(x, h1, mode='full')
conv_h1_x = convolve2d(h1, x, mode='full')
conmutativa = np.array_equal(conv_x_h1, conv_h1_x)
print("Conmutativa ",conmutativa)

# Propiedad Distributiva: x * (h1 + h2) = x * h1 + x * h2
sum_h1_h2 = h1 + h2
conv_x_sum = convolve2d(x, sum_h1_h2, mode='full')
conv_x_h1 = convolve2d(x, h1, mode='full')
conv_x_h2 = convolve2d(x, h2, mode='full')
distributiva = np.array_equal(conv_x_sum, conv_x_h1 + conv_x_h2)
print("Distributiva ",distributiva)

# Propiedad Asociativa: x * (h1 * h2) = (x * h1) * h2
conv_h1_h2 = convolve2d(h1, h2, mode='full')
conv_x_h1_h2 = convolve2d(x, conv_h1_h2, mode='full')
conv_x_h1 = convolve2d(x, h1, mode='full')
conv_x_h1_then_h2 = convolve2d(conv_x_h1, h2, mode='full')
asociativa = np.array_equal(conv_x_h1_h2, conv_x_h1_then_h2)
print("Asociativa ",asociativa)

Conmutativa  True
Distributiva  True
Asociativa  True


### Ejercicio 4.

In [26]:
def load_image(filepath, grayscale=True):
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
    return image
def gaussian_kernel(size=9, sigma=2):
    kernel_range = np.linspace(-(size // 2), size // 2, size)
    
    gauss_1d = np.exp(-0.5 * (kernel_range / sigma)**2)
    gauss_1d = gauss_1d / np.sum(gauss_1d)  # Normalizar el kernel 1D
    
    # Generar el kernel 2D tomando el producto externo del kernel 1D
    gauss_2d = np.outer(gauss_1d, gauss_1d)
    
    return gauss_2d

def create_filter(filter_type, size=3):
    if filter_type == "mean":
        # Filtro de media (matriz promedio)
        return np.ones((size, size)) / (size * size)
    elif filter_type == "gaussian":
        # Filtro gaussiano 3x3 o 9x9 (aproximación)
        if size == 3:
            return np.array([[1, 2, 1],
                             [2, 4, 2],
                             [1, 2, 1]]) / 16
        elif size == 9:
            return gaussian_kernel()
    return None

# Aplicar los filtros de convolución
def apply_filter(image, filter_type, size=3):
    if filter_type in ["mean", "gaussian"]:
        # Aplicar filtro de media o gaussiano usando convolución 2D
        kernel = create_filter(filter_type, size)
        return convolution_2d(image,kernel)
    elif filter_type == "median":
        # Aplicar filtro de mediana usando scipy.ndimage
        return median_filter(image, size=size)
    elif filter_type == "maximum":
        # Aplicar filtro de máximo usando scipy.ndimage
        return maximum_filter(image, size=size)
    elif filter_type == "minimum":
        # Aplicar filtro de mínimo usando scipy.ndimage
        return minimum_filter(image, size=size)
    else:
        raise ValueError("Tipo de filtro no soportado.")

# Mostrar las imágenes procesadas
def plot_results(original, filtered_images, filter_names, size):
    plt.figure(figsize=(15, 8))
    plt.subplot(2, 4, 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original")
    plt.axis('off')
    
    # Mostrar cada imagen filtrada
    for i, (filtered, name) in enumerate(zip(filtered_images, filter_names)):
        plt.subplot(2, 4, i + 2)
        plt.imshow(filtered, cmap='gray')
        plt.title(f"{name} ({size}x{size})")
        plt.axis('off')
    plt.show()


# Cargar la imagen (puedes cambiar la ruta a tu imagen)
image_path = 'cameraman.jpg'  # Cambiar a la ruta de tu imagen
image = load_image(image_path)

# Aplicar los filtros 3x3 y 9x9
for size in [3, 9]:
    # Aplicar cada uno de los filtros
    mean_filtered = apply_filter(image, "mean", size=size)
    gaussian_filtered = apply_filter(image, "gaussian", size=size)
#         median_filtered = apply_filter(image, "median", size=size)
#         maximum_filtered = apply_filter(image, "maximum", size=size)
#         minimum_filtered = apply_filter(image, "minimum", size=size)

    # Mostrar resultados
    filtered_images = [mean_filtered, gaussian_filtered]
    filter_names = ["Media", "Gaussiano", "Mediana", "Máximo", "Mínimo"]
    plot_results(image, filtered_images, filter_names, size)

IndexError: index 7 is out of bounds for axis 1 with size 7